# Setup Paths

In [ ]:
import os
import re

In [ ]:
from PIL import Image

In [ ]:
path = "Enter the path to the model directory: "
path = "C:\\Users\\aviss\\Documents\\AdvDSP\\FindingNemo\\Nemo Model"

In [ ]:
# yolo_class_list_file = os.path.join(path, "Enter the path to the file that has the yolo classes (typically classes.txt): ")

folder_holding_all_annotations = os.path.join(path,"uot32","dataset")

folder_holding_yolo_annotations = os.path.join(path,"uot32","dataset","SeaTurtle2")
file_holding_yolo_annotations = os.path.join(path,"uot32","dataset","SeaTurtle2", "groundtruth_rect.txt")
yolo_class_list_file = os.path.join(path,"uot32","classes.txt")

# Get List of Classes

In [ ]:
# Get a list of all the classes used in the yolo format
with open(yolo_class_list_file) as f:
    yolo_classes = f.readlines()
array_of_yolo_classes = [x.strip() for x in yolo_classes]

In [ ]:
array_of_yolo_classes

In [ ]:
# Description of Yolo Format values
# 15 0.448743 0.529142 0.051587 0.021081
# class_number x_yolo y_yolo yolo_width yolo_height

def is_number(n):
  try:
    float(n)
    return True
  except ValueError:
    return False

# Convert Data

In [ ]:
# gets the names of all subfolders under the dataset directory 
classes = [f.name for f in os.scandir(folder_holding_all_annotations) if f.is_dir()]

print("classes")
print(classes)

for c in classes:
  print(c) 
  counter = 1

  file_holding_yolo_annotations = os.path.join(folder_holding_all_annotations, c, "groundtruth_rect.txt")

  the_file = open(file_holding_yolo_annotations, 'r')
  all_lines = the_file.readlines()

  xml_path = os.path.join(folder_holding_all_annotations, c, 'XML')
  if not os.path.exists(xml_path):
    # If an XML folder does not already exist, make one
    os.makedirs(xml_path)
  else: 
    print("XML folder exists")

  for line in all_lines: 
    image_name = str(counter) + ".jpg"
    image_path = os.path.join(folder_holding_all_annotations, c, "img", image_name)

    counter += 1

    print("Converting " + c + str(counter))
    
    # If the image name is the same as the yolo filename
    # then we did NOT find an image that matches, and we will skip this code block
    orig_img = Image.open(image_path) # open the image
    image_width = orig_img.width
    image_height = orig_img.height

    # Start the XML file
    with open(image_path.replace('jpg', 'xml').replace('img', 'XML'), 'w') as f:
      f.write('<annotation>\n')
      f.write('\t<folder>XML</folder>\n')
      f.write('\t<filename>' + image_name + '</filename>\n')
      f.write('\t<path>' + os.getcwd() + os.sep + image_name + '</path>\n')
      f.write('\t<source>\n')
      f.write('\t\t<database>Unknown</database>\n')
      f.write('\t</source>\n')
      f.write('\t<size>\n')
      f.write('\t\t<width>' + str(image_width) + '</width>\n')
      f.write('\t\t<height>' + str(image_height) + '</height>\n')
      f.write('\t\t<depth>3</depth>\n') # assuming a 3 channel color image (RGB)
      f.write('\t</size>\n')
      f.write('\t<segmented>0</segmented>\n')
    
      # regex to find the numbers in each line of the text file
      yolo_array = re.split("\t", line.rstrip()) # remove any extra space from the end of the line

      # here the class is taken from the name of the folder
      yolo_array_contains_only_digits = True

      # make sure the array has the correct number of items
      if len(yolo_array) == 4:
        for each_value in yolo_array:
          # If a value is not a number, then the format is not correct, return false
          if not is_number(each_value):
            yolo_array_contains_only_digits = False
        
        if yolo_array_contains_only_digits:
          # assign the variables
          #class_number = int(yolo_array[0])
          #object_name = array_of_yolo_classes[class_number]
          x_min = float(yolo_array[0])
          y_min = float(yolo_array[1])
          width = float(yolo_array[2])
          height = float(yolo_array[3])

          # Convert Yolo Format to Pascal VOC format
          box_width = width
          box_height = height

          x_max = x_min + width 
          y_max = y_min + height 

          # write each object to the file
          f.write('\t<object>\n')
          f.write('\t\t<name>' + c + '</name>\n')
          f.write('\t\t<pose>Unspecified</pose>\n')
          f.write('\t\t<truncated>0</truncated>\n')
          f.write('\t\t<difficult>0</difficult>\n')
          f.write('\t\t<bndbox>\n')
          f.write('\t\t\t<xmin>' + str(x_min) + '</xmin>\n')
          f.write('\t\t\t<ymin>' + str(y_min) + '</ymin>\n')
          f.write('\t\t\t<xmax>' + str(x_max) + '</xmax>\n')
          f.write('\t\t\t<ymax>' + str(y_max) + '</ymax>\n')
          f.write('\t\t</bndbox>\n')
          f.write('\t</object>\n')

      # Close the annotation tag once all the objects have been written to the file
      f.write('</annotation>\n')
      f.close() # Close the file